In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("refactored_foods.csv")
df = df.iloc[:841,:]
df

In [ ]:
df = df.drop('image nr', axis=1)
df = df[df['Food vs nonfood'] == "Food"]
df = df.drop('Food vs nonfood', axis=1)
df = df.drop('State of food', axis=1)
df = df[df['Food classification'] != 'Unappealing']
df = df[df['Food classification'] !=  'African']
#df = df.dropna(subset=['S'],how='all')
df.reset_index(drop=True,inplace=True)
#df = df.iloc[:,0:27]
df['Food classification'].value_counts()

In [ ]:
def standardize_classification(item):
    if (not isinstance(item, str)) and (math.isnan(float(item))):
        return item
    elif item == "Universal":
        return 0
    elif item == "Western":
        return 1
    elif item == "Asian":
        return 2

def standardize_natural(item):
    if (not isinstance(item, str)) and (math.isnan(float(item))):
        return item
    elif item == "Natural":
        return 1
    else:
        return 0
    
def standardize_flavor(item):
    if (not isinstance(item, float)) and (not isinstance(item, str)) and (math.isnan(float(item))):
        return item
    elif item == "Savory":
        return 1
    else:
        return 0
    
def standardize_complexity(item):
    if (not isinstance(item, str)) and (math.isnan(float(item))):
        return item
    elif (item == "Complex") or (item == "complex"):
        return 10
    elif (int(item) >= 10):
        return 10
    else:
        return item

In [ ]:
df['Food classification'] = df['Food classification'].apply(lambda x: standardize_classification(x))
df['Natural vs transformed'] = df['Natural vs transformed'].apply(lambda x: standardize_natural(x))
df['Sweet vs savory'] = df['Sweet vs savory'].apply(lambda x: standardize_flavor(x))
df['Complexity (amount, number of items)'] = df['Complexity (amount, number of items)'].apply(lambda x: standardize_complexity(x))
df

In [ ]:
universalIdx = df.index[df['Food classification'] == 0].tolist()
westernIdx = df.index[df['Food classification'] == 1].tolist()
easternIdx = df.index[df['Food classification'] == 2].tolist()

In [ ]:
temp = df.dropna()
temp = temp.drop('Description', axis=1)

In [ ]:
model = LogisticRegression()

for i in range(0,5):

    X = temp.drop(columns=['Food classification'])
    Y = temp['Food classification']
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

    model.fit(x_train, y_train)
    print("Accuracy of LR Model: ", model.score(x_test, y_test))

In [ ]:
temp = df
temp = df[df['Average_Health_JP'].notna() & df['St._dev_Health_JP'].notna()]
temp

imp = IterativeImputer(max_iter=5, n_nearest_features=10, sample_posterior=True, random_state=0)
imp.fit(temp.iloc[:,2:])
newDf = imp.transform(df.iloc[:,2:])


In [ ]:
df2 = pd.DataFrame(newDf)
df2.head()

In [ ]:
classif = df["Food classification"]
df2.insert(0, 'Food classification', classif)

df2.iloc[:,2] = df2.loc[:,2].apply(lambda x: round(abs(x)))
df2

In [ ]:
for i in range(0,5):

    X = df2.drop(columns=['Food classification'])
    Y = df2['Food classification']
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

    model.fit(x_train, y_train)
    
    print("Accuracy of LR Model: ", model.score(x_test, y_test))

In [ ]:
y_pred = model.predict(x_test)

probs_y = model.predict_proba(x_test)
probs_y = np.round(probs_y, 2)

pred_perc = pd.DataFrame(probs_y)
pred_perc.columns = ['Universal%', 'Western%', 'Asian%']

display(pred_perc)
display(pd.DataFrame(y_test))

In [ ]:
df

In [ ]:
reduced_data = PCA(n_components=2).fit_transform(df2)
pcaDf = pd.DataFrame(reduced_data)
pcaDf

In [ ]:
universalCords = pcaDf.iloc[universalIdx]
westernCords = pcaDf.iloc[westernIdx]
easternCords = pcaDf.iloc[easternIdx]

In [ ]:
# Code Reference: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html

kmeans = KMeans(init="k-means++", n_clusters=3, n_init=4, max_iter=300)
y_kmeans = kmeans.fit_predict(reduced_data)

plt.scatter(reduced_data[y_kmeans==0, 0], reduced_data[y_kmeans==0, 1], s=5, c='red', label ='Cluster 1')
plt.scatter(reduced_data[y_kmeans==1, 0], reduced_data[y_kmeans==1, 1], s=5, c='blue', label ='Cluster 1')
plt.scatter(reduced_data[y_kmeans==2, 0], reduced_data[y_kmeans==2, 1], s=5, c='green', label ='Cluster 1')

#plt.plot(reduced_data[:, 0], reduced_data[:, 1], "k.", markersize=2)

centroids = kmeans.cluster_centers_

plt.scatter(
   centroids[:, 0],
   centroids[:, 1],
   marker="x",
   s=169,
   linewidths=3,
   color="black",
   zorder=10,
)

plt.xticks(visible = True)
plt.xticks(visible = True)
plt.show()

In [ ]:
plt.scatter(universalCords.iloc[:,0] , universalCords.iloc[:,1] , color = 'red', s=5)
plt.scatter(westernCords.iloc[:,0] , westernCords.iloc[:,1] , color = 'green', s=5)
plt.scatter(easternCords.iloc[:,0] , easternCords.iloc[:,1] , color = 'blue', s=5)
plt.scatter(
   centroids[:, 0],
   centroids[:, 1],
   marker="x",
   s=169,
   linewidths=3,
   color="black",
   zorder=10,
)
plt.show()